In [15]:
from deap import base, creator, tools
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

全局参数

In [16]:
TMAX = 100          # 为演示缩短长度，实际可设1000
K = 10
P = 3

D_total = 1e8
Cap_rocket = 150
Cap_lift = 179000
L_max = 800

Cost_rocket = 500000
Cost_lift = 100000

初始化DEAP

In [17]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# 火箭发射次数 (整数)
toolbox.register("attr_R", lambda: random.randint(0, L_max))
# 电梯运输量 (连续)
toolbox.register("attr_E", lambda: random.uniform(0, Cap_lift))

def init_individual():
    """保证初始化总运输量大约 D_total / TMAX"""
    ind = []
    avg_rocket_per_year = D_total / TMAX / 2 / Cap_rocket / K
    avg_lift_per_year = D_total / TMAX / 2 / P

    for _ in range(K*TMAX):
        val = max(1, int(random.gauss(avg_rocket_per_year, avg_rocket_per_year*0.5)))
        ind.append(min(val, L_max))

    for _ in range(P*TMAX):
        val = random.gauss(avg_lift_per_year, avg_lift_per_year*0.5)
        val = max(0, min(val, Cap_lift))
        ind.append(val)

    return creator.Individual(ind)

toolbox.register("individual", init_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


fitness分配

In [18]:
def assign_fitness(pop, w1=0.5, w2=0.5):
    ZC_list = []
    ZT_list = []

    for ind in pop:
        R = np.array(ind[:K*TMAX]).reshape(K, TMAX)
        E = np.array(ind[K*TMAX:]).reshape(P, TMAX)

        # 总成本
        ZC = Cost_rocket * np.sum(R) + Cost_lift * np.sum(E)

        # 总工期
        transported = 0
        ZT = TMAX
        for t in range(TMAX):
            R_t = np.sum(R[:, t]) * Cap_rocket
            E_t = np.sum(E[:, t])
            transported += R_t + E_t
            if transported >= D_total:
                ZT = t + 1
                break

        # 罚函数: 总运输量不足时增加 fitness
        penalty = max(0, D_total - transported) * 1e-3  # 连续罚函数

        ind.ZC = ZC
        ind.ZT = ZT
        ind.penalty = penalty

        ZC_list.append(ZC)
        ZT_list.append(ZT)

    # 种群内标准化
    std_C = np.std(ZC_list) + 1e-6
    std_T = np.std(ZT_list) + 1e-6

    for ind in pop:
        fitness_val = w1 * ind.ZC / std_C + w2 * ind.ZT / std_T + ind.penalty
        ind.fitness.values = (fitness_val,)

遗传算子

In [19]:
toolbox.register("mate", tools.cxTwoPoint)

def mutate_bound(ind, mu_R=0, sigma_R=100, mu_E=0, sigma_E=5e4, indpb=0.1):
    for i in range(len(ind)):
        if random.random() < indpb:
            if i < K*TMAX:
                ind[i] += int(round(random.gauss(mu_R, sigma_R)))
                ind[i] = max(0, min(L_max, ind[i]))
            else:
                ind[i] += random.gauss(mu_E, sigma_E)
                ind[i] = max(0, min(Cap_lift, ind[i]))
    return ind,

toolbox.register("mutate", mutate_bound)
toolbox.register("select", tools.selTournament, tournsize=3)

In [13]:
def sweep_weights(toolbox, pop_size=50, NGEN=40, w1_values=50):
    """
    对权重 w1 从0到1取等距值，运行GA，返回每个权重下每一代的最优解。
    
    参数：
        toolbox: DEAP 注册好的 toolbox（含 individual, population, mate, mutate, select）
        pop_size: 种群大小
        NGEN: 迭代代数
        w1_values: 权重 w1 的等分数（整数或列表）
        
    返回：
        df: pandas DataFrame，列 = ['w1','w2','ZC','ZT','Z']
    """
    
    # 如果 w1_values 是整数，就生成等距值
    if isinstance(w1_values, int):
        w1_list = np.linspace(0, 1, w1_values)
    else:
        w1_list = np.array(w1_values)
    
    records = []  # 用来存储每个权重下每代最优解

    for w1 in w1_list:
        w2 = 1 - w1
        
        # 初始化种群
        pop = toolbox.population(n=pop_size)
        
        # 第一次计算 fitness
        assign_fitness(pop, w1=w1, w2=w2)  # 需要你的 assign_fitness 函数支持 w1, w2 参数
        
        for gen in range(NGEN):
            # 选择
            offspring = toolbox.select(pop, len(pop))
            offspring = list(map(toolbox.clone, offspring))
            
            # 交叉
            for c1, c2 in zip(offspring[::2], offspring[1::2]):
                if np.random.random() < 0.7:
                    toolbox.mate(c1, c2)
                    del c1.fitness.values, c2.fitness.values
            
            # 变异
            for ind in offspring:
                if np.random.random() < 0.2:
                    toolbox.mutate(ind)
                    del ind.fitness.values
    
            # 重新计算 fitness
            assign_fitness(offspring, w1=w1, w2=w2)
            pop[:] = offspring
            
            # 找每代最优解
            best = tools.selBest(pop, 1)[0]
            
        # 保存到 records
        records.append({
            'w1': w1,
            'w2': w2,
            'ZC': best.ZC,
            'ZT': best.ZT,
            'Z': best.fitness.values[0],
        })
    
    # 转换为 DataFrame
    df = pd.DataFrame(records)
    return df

In [14]:
df_results = sweep_weights(toolbox, pop_size=50, NGEN=50, w1_values=50)

df_results.to_csv(r"D:\USELESS\数据分析学习\数学建模学习\2026美赛\B\prob01_result1.csv")

main函数入口

In [20]:
def main():
    pop = toolbox.population(n=50)
    NGEN = 50
    CXPB = 0.7
    MUTPB = 0.2

    assign_fitness(pop)

    for gen in range(NGEN):
        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))

        # 交叉
        for c1, c2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(c1, c2)
                del c1.fitness.values, c2.fitness.values

        # 变异
        for ind in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(ind)
                del ind.fitness.values

        assign_fitness(offspring)
        pop[:] = offspring

        best = tools.selBest(pop, 1)[0]
        total_transported = np.sum(best[:K*TMAX])*Cap_rocket + np.sum(best[K*TMAX:])
        print(f"Gen {gen}: Z={best.fitness.values[0]:.3e}, "
              f"ZC={best.ZC:.3e}, ZT={best.ZT}, TotalTransport={total_transported:.1e}")

    return tools.selBest(pop, 1)[0]

运行GA

In [21]:
best_ind = main()

R = np.array(best_ind[:K*TMAX]).reshape(K, TMAX)
E = np.array(best_ind[K*TMAX:]).reshape(P, TMAX)

ROCKET = pd.DataFrame(R)
ELEVATOR = pd.DataFrame(E)
ROCKET.to_excel((r"D:\USELESS\数据分析学习\数学建模学习\2026美赛\B\ROCKET1.xlsx"))
ELEVATOR.to_excel((r"D:\USELESS\数据分析学习\数学建模学习\2026美赛\B\ELEVATOR1.xlsx"))
print("最终总运输量 =", np.sum(R)*Cap_rocket + np.sum(E))
print('火箭运输量 = ', np.sum(R)*Cap_rocket)
print('太空电梯运输量 = ', np.sum(E))


Gen 0: Z=5.000e+07, ZC=4.554e+12, ZT=100, TotalTransport=9.5e+07
Gen 1: Z=5.000e+07, ZC=4.584e+12, ZT=100, TotalTransport=9.6e+07
Gen 2: Z=5.000e+07, ZC=4.586e+12, ZT=100, TotalTransport=9.6e+07
Gen 3: Z=5.000e+07, ZC=4.586e+12, ZT=100, TotalTransport=9.6e+07
Gen 4: Z=5.000e+07, ZC=4.587e+12, ZT=100, TotalTransport=9.7e+07
Gen 5: Z=5.000e+07, ZC=4.587e+12, ZT=100, TotalTransport=9.7e+07
Gen 6: Z=5.000e+07, ZC=4.587e+12, ZT=100, TotalTransport=9.7e+07
Gen 7: Z=5.000e+07, ZC=4.588e+12, ZT=100, TotalTransport=9.7e+07
Gen 8: Z=5.000e+07, ZC=4.588e+12, ZT=100, TotalTransport=9.7e+07
Gen 9: Z=5.000e+07, ZC=4.588e+12, ZT=100, TotalTransport=9.7e+07
Gen 10: Z=5.000e+07, ZC=4.588e+12, ZT=100, TotalTransport=9.7e+07
Gen 11: Z=5.000e+07, ZC=4.588e+12, ZT=100, TotalTransport=9.7e+07
Gen 12: Z=5.000e+07, ZC=4.589e+12, ZT=100, TotalTransport=9.7e+07
Gen 13: Z=5.000e+07, ZC=4.589e+12, ZT=100, TotalTransport=9.7e+07
Gen 14: Z=5.000e+07, ZC=4.596e+12, ZT=100, TotalTransport=9.8e+07
Gen 15: Z=5.000e+07,